# `BootstrapImages Notebook`
This file is for pasting game character crops onto game backgrounds. The current implementation
either places objects randomly around the background or normally distributed around a central point.
Objects are placed in fully visible areas, and object occlusion is not taken into account <br>
 ### `Planned Improvements`
- Foreground object-crop placement like minimap, health
- Random noise. Can be added before training, or right after generation
- Account for occluded objects using IoU. 
- After object detection,animation classification! 

In [172]:
import numpy as np
from PIL import ImageDraw, ImageFont
import PIL.Image
from PIL import ImageFilter
from shapely.geometry import Polygon
import random
import os
from os import listdir, path
from IPython.display import Image 

In [184]:
output_size = 1000
centre_padding = 400 # 400 is good
centre_stdev_min_max = [250, 350] # stdev x y should be less than centre padding so that coordinates are within backgnd! negative coordinates map to zero in image lib
random_padding = [200, 200]
rotate_min_max = [-30,30]
cropped_img_size_offset = [-5, -5]

In [185]:
# Global variables
scales = {'tower' : [0.6, 0.9],
         'creep'  : [1.2, 2.0], 
         'champ'  : [1.5, 2.5]
         }

            # Red structures
classes = { 'rtower' :0,
            'rinhib' :0,
            'rnexus' :0, 
           
            'btower' :1,
            'binhib' :1,
            'bnexus' :1,
           
            'rmelee' :2,
            'rranged':2,
            'rseiged':2,
            'rsuper' :2,
           
            'bmelee' :3,
            'branged':3,
            'bseiged':3,
            'bsuper' :3,
           
            'champ_garen'  :4,
            'champ_morgana':5, 
            'champ_nasus'  :6,
          }

creeps_min_max = [0, 10]
champs_num = 3
champs_min_max = [0, champs_num]

n_classes = len(classes)
path_bckgnds = 'map_screenshots/'
path_creep = 'creeps_cropped/'
path_rtowers = 'towers_cropped/red/'
path_btowers = 'towers_cropped/blue/'
path_garen = 'Champions_cropped/champions/Garen/'
path_nasus = 'Champions_cropped/champions/Nasus/'
path_morgana = 'Champions_cropped/champions/Morgana/'
path_save_synthetic_imgs = 'output/imgs/'
path_save_labels = 'output/labels/'
save_prefix = 'lol_'

In [186]:
def get_files_in_dir_as_dict(path, filt='.csv'):
    ''' Get all files from path. Returns a dict of folder + path'''
    assert os.path.exists(path), "The path {} was not found!".format(path)
    f = dict()
    for (dirpath, dirnames, filenames) in os.walk(path):
        files = [os.path.join(dirpath,f) for f in filenames if filt in f]
        if len(files) > 0:
            f[dirpath] = files
    return f
def get_files_in_dir_as_list(path, filt='.csv'):
    ''' Wrapper for the function above. Gets all files in a directory as a list'''
    f_dict = get_files_in_dir_as_dict(path, filt)
    return [k for key in f_dict for k in f_dict[key]]

In [187]:
def apply_noise_pixel(pixel, noise=(10,10,10)):
    """
    This funciton applies random noise to the rgb values of a pixel (R,G,B)
    """
    R = max(0, min(255, pixel[0] + random.randint(-noise[0], noise[0])))
    G = max(0, min(255, pixel[1] + random.randint(-noise[1], noise[1])))
    B = max(0, min(255, pixel[2] + random.randint(-noise[2], noise[2])))
    A = pixel[3]
    return (R, G, B, A)

def apply_noise_img(img, noise=(10,10,10), img_portion=0.5):
    ''' Applies noise to image pixels of portion size x'''
    w, h = img.size
    img_data = list(img.getdata())
    len_data = len(img_data)
    idxs = random.sample(range(len_data), k=int(sample*len_data))
    for x in idxs:
        img_data[x] = apply_noise_pixel(img_data[x], noise)
    img.putdata(img_data)        
    return img

def random_rescale(og_size, min_max_scale):
    """
    Rescale size according to random scale factor depending on image class
    """
    #scale_min, scale_max = min_max_scale
    #print(og_size, min_max_scale)
    scale_factor = random.uniform(*min_max_scale)
    return int(og_size[0]*scale_factor), int(og_size[1]*scale_factor)

def get_obj_scale_factor(obj_file_name):
        if('creep' in obj_file_name):
            return scales['creep']
        if('tower' in obj_file_name):
            return scales['tower']
        if('ampion' in obj_file_name):
            return scales['champ']
        
def get_obj_corners(pos, obj_size, bckgrnd_size=(1920, 1080)):
    w, h = obj_size
    bw, bh = bckgrnd_size
    min_x = int(max(pos[0], 0))
    min_y = int(max(pos[1], 0))
    min_x = min(min_x, bckgrnd_size[0] - obj_size[0])
    min_y = min(min_y, bckgrnd_size[1] - obj_size[1])
    max_x = min_x + obj_size[0]
    max_y = min_y + obj_size[1]
    return min_x, min_y, max_x, max_y 

def get_obj_center_pos(pos, size):
    return int(pos[0] + size[0]/2), int(pos[1] + size[1]/2)

def create_yolo_label_str(cpos, size, img_class, bckgnd_size = (1920, 1080)): 
    return "{} {} {} {} {}\n".format(img_class, cpos[0]/bckgnd_size[0],cpos[1]/bckgnd_size[1], 
                                   size[0]/bckgnd_size[0], size[1]/bckgnd_size[1])

def get_box(pos, size):
    box = [[pos[0],           pos[1]          ], # top-left
           [pos[0] + size[0], pos[1]          ],
           [pos[0] + size[0], pos[1] + size[1]],
           [pos[0],           pos[1] + size[1]]
          ]
    return box

def calculate_iou(pos1, size1, pos2, size2):
    '''
    Calculate IoU in oder to get percentage of occlusion of 1 image by another.
    If occlued by > 0.80%, discard
    '''
    poly_1 = Polygon(get_box(pos1, size1))
    poly_2 = Polygon(get_box(pos2, size2))
    iou = poly_1.intersection(poly_2).area / poly_1.union(poly_2).area
    return iou
        
def draw_bound_box(curr_bckgnd, x1,y1,x2,y2,obj_cpos,ob_name, curr_objects_ls):
    box = ImageDraw.Draw(curr_bckgnd) 
    box.rectangle([(x1,y1), (x2,y2)], outline="red")
    box.text((obj_cpos[0]-11, obj_cpos[1]-22), text='+', fill='red', font=ImageFont.truetype("arial.ttf", 40), )
    box.text((x1-11, y1-22), text='+', fill='green', font=ImageFont.truetype("arial.ttf", 40))
    #box.text((obj_cpos[0], y1), text='+', fill='green', font=ImageFont.truetype("arial.ttf", 40))
    #box.text((x1, obj_cpos[1]), text='+', fill='green', font=ImageFont.truetype("arial.ttf", 40))
    ob_name = path.basename(curr_objects_ls[i][0])
    box.text((x1, y1), text='{}\n{}'.format(ob_name, obj_cpos), stroke_fill='red', font=ImageFont.truetype("arial.ttf", 15))
     

In [188]:
# Get images by class
f_rcreeps = get_files_in_dir_as_list(path_creep,  'Chaos')
f_rcreeps = [x for x in f_rcreeps if 'eath' not in x] # Filter death animation. Affects boundimg box size
f_bcreeps = get_files_in_dir_as_list(path_creep,  'Order')
f_bcreeps = [x for x in f_bcreeps if 'eath' not in x]
f_morgana = get_files_in_dir_as_list(path_morgana,'png')
f_nasus   = get_files_in_dir_as_list(path_nasus, 'png')
f_garen   = get_files_in_dir_as_list(path_garen, 'png')
f_rtowers = get_files_in_dir_as_list(path_rtowers, 'png')
f_btowers = get_files_in_dir_as_list(path_btowers, 'png')
f_bckgnds   = get_files_in_dir_as_list(path_bckgnds, 'png')

# Add images to array by class name
objects = dict()
objects[classes['rtower']] = f_rtowers
objects[classes['rmelee']] = f_rcreeps
objects[classes['btower']] = f_btowers
objects[classes['bmelee']] = f_bcreeps
objects[classes['champ_nasus']] = f_nasus
objects[classes['champ_garen']] = f_garen
objects[classes['champ_morgana']] = f_morgana

# Champs
champs = [champ for champ in classes if 'champ' in champ]

In [189]:
os.makedirs(path_save_labels,exist_ok=True)
os.makedirs(path_save_synthetic_imgs, exist_ok=True)

In [190]:
PATH_IDX, CLASS_IDX, POS_IDX,  CPOS_IDX, SIZE_IDX = 0,1,2,3,4
# Ok, I'm in a hurry, So i'm going to do this real - quick and messy. Don't judge me!

for img_num in range(output_size):
    # Get a random number of creeps between the min and max values
    curr_objects = dict()
    rcreeps = random.choices(f_rcreeps, k=random.randint(creeps_min_max[0], creeps_min_max[1]))
    bcreeps = random.choices(f_bcreeps, k=random.randint(creeps_min_max[0], creeps_min_max[1]))
    curr_objects[classes['rmelee']] = rcreeps
    curr_objects[classes['bmelee']] = bcreeps
    
    # Towers. Always one tower in an image. Randomly chose between red and blue, p=0.5
    if(random.random() > 0.5):
        tower = random.choice(f_rtowers)
        curr_objects[classes['rtower']] = [tower]
    else:
        tower = random.choice(f_btowers)
        curr_objects[classes['btower']] = [tower]
    
    # Select champion names randomly
    champ_names = random.choices(champs, k=random.randint(champs_min_max[0], champs_min_max[1]))
    
    # Select champ images randomly from fine names. 
    # If same champion is chosen more than once, value is overwritten
    for champ in champ_names:
        file = random.choice(objects[classes[champ]])
        #curr_champs.append(file)
        curr_objects[classes[champ]] = [file]
        
    # Dict to list of (filename,class, [posx, posy], [cposx, cposy], [w, h])
    curr_objects_ls = [(val, ob_class, [int, int], [int, int],  [int, int])  
                       for ob_class in curr_objects 
                       for val in curr_objects[ob_class]]
    
    # shuffle the dict
    random.shuffle(curr_objects_ls)
    
    # Get background
    curr_bckgnd =  PIL.Image.open(random.choice(f_bckgnds))
    bck_w, bck_h = curr_bckgnd.size
    # Make sure the image is 1920x1080
    assert (bck_w == 1920 and bck_h == 1080), "Error! Background image is {}x{}. It has to be 1920x1080".format(bck_w, bck_h) 

    # Randomly chose between random and normally distributed placement of images around central point
    # Get random central corrdinate. All images will be clustered around central corrdinate
    central_pos = (random.randint(centre_padding, bck_w-1-centre_padding), 
                   random.randint(centre_padding, bck_h-1-centre_padding))
    
    # Get stdev for placing images around central point
    stdev = random.randint(*centre_stdev_min_max)
    yolo_labels = ''
    for i, obj in zip(range(len(curr_objects_ls)), curr_objects_ls):
        if(random.random() < 0.7):
            x1 = int(np.random.normal(loc=central_pos[0], scale=stdev))
            y1 = int(np.random.normal(loc=central_pos[1], scale=stdev))
        else:
            # get a random pos on the map
            x1 = np.random.randint(0, bck_w-random_padding[0])
            y1 = np.random.randint(0, bck_h-random_padding[1])
            

        # Get the actual image
        curr_im = PIL.Image.open(curr_objects_ls[i][PATH_IDX]).convert("RGBA")
        
        # Random rotate
        rot_val = random.randint(*rotate_min_max)
        curr_im = curr_im.rotate(rot_val, expand=1) # expand = 1 to change image size 
        
        # Add size to data array. w, h are index 3
        scale_factor = get_obj_scale_factor(curr_objects_ls[i][PATH_IDX])
        rescaled_size = random_rescale(curr_im.size, scale_factor)
        
        # Rescale image 
        curr_im = curr_im.resize(rescaled_size)
    
        #rescaled_size = rescaled_size[0] + cropped_img_size_offset[0],  rescaled_size[1] + cropped_img_size_offset[1]
        x1, y1, x2, y2 = get_obj_corners((x1, y1), rescaled_size) # Object may fall out of background image! Hence get actual corners in image
        rescaled_size = abs(x1 - x2), abs(y1- y2)
        
        # Cropped image size is larger than actual object. 
        # Reduce bounding box size by inc xmin, ymin and reducing xmax, ymax by a percentage of img size
        bbox_pcnt = 0.1
        x_redxn = int(rescaled_size[0]*bbox_pcnt)
        y_redxn = int(rescaled_size[1]*bbox_pcnt)
        x1_bbox, y1_bbox, x2_bbox, y2_bbox = x1 + x_redxn, y1 + y_redxn, x2 - x_redxn, y2 - y_redxn
        size_bbox = abs(x1_bbox - x2_bbox), abs(y1_bbox- y2_bbox)
        # Get object center position, add to array index 3
        obj_cpos = get_obj_center_pos((x1, y1), rescaled_size)
        
        # Add the obj data to array
        curr_obj_class = curr_objects_ls[i][CLASS_IDX]
        curr_objects_ls[i][POS_IDX][0] =  x1_bbox
        curr_objects_ls[i][POS_IDX][1] =  y1_bbox
        curr_objects_ls[i][SIZE_IDX][0] = size_bbox[0]
        curr_objects_ls[i][SIZE_IDX][1] = size_bbox[1]
        curr_objects_ls[i][CPOS_IDX][0] = obj_cpos[0]
        curr_objects_ls[i][CPOS_IDX][1] = obj_cpos[1]
        
        # TODO: Add random noise to image the end
        curr_bckgnd.paste(curr_im, box=(x1, y1), mask=curr_im)
        
        # Create yolo label
        yolo_labels += create_yolo_label_str(obj_cpos,size_bbox, curr_obj_class, (bck_w, bck_h))
        # Debugging. Draw box around image
        #draw_bound_box(curr_bckgnd,x1_bbox, y1_bbox, x2_bbox, y2_bbox,obj_cpos,ob_name, curr_objects_ls)
    #display(curr_bckgnd)
    
    # Save image 
    save_name = path_save_synthetic_imgs + save_prefix + str(img_num) +'.jpg'
    save_lbl = path_save_labels + save_prefix + str(img_num) +'.txt'
    
    with open(save_lbl, 'w+') as f:
        f.write(yolo_labels)
    curr_bckgnd.save(save_name)


In [192]:
import torch

In [193]:
torch.__version__

'1.8.0+cpu'